In [10]:
import numpy as np
import pandas as pd
import os

#import microsim modules that we need in order to run a trial
from microsim.trials.trial_type import TrialType
from microsim.trials.trial_description import NhanesTrialDescription
from microsim.risk_factor import StaticRiskFactorsType, DynamicRiskFactorsType
from microsim.person_filter_factory import PersonFilterFactory
from microsim.trials.trial_outcome_assessor_factory import TrialOutcomeAssessorFactory
from microsim.trials.trial import Trial
from microsim.population_factory import PopulationFactory
from microsim.outcome import OutcomeType
from microsim.person_filter import PersonFilter
from microsim.person_factory import PersonFactory

#switch to the Microsim directory
microsimDir = "/Users/deligkaris.1/OneDrive - The Ohio State University Wexner Medical Center/MICROSIM/CODE/microsim"
os.chdir(microsimDir)

Specify NHANES population variables

In [14]:
year=2017
duration=10
durationEndOfTime=200
nhanesWeights=True
popSize=100000
nWorkers=5

### Incidence and prevalence

Incidence and prevalence right after the creation of the population, the beginning of time, should be the same

In [3]:
pop = PopulationFactory.get_nhanes_population(n=popSize, year=year, nhanesWeights=nhanesWeights)

pop.print_outcome_incidence_prevalence(outcomeType=OutcomeType.EPILEPSY)

                          -----------------------------------------------------
                          epilepsy incidence rate (first incidence only)
                          -----------------------------------------------------
                    age   rate
                  15-19  0.002
              15.0-19.0  0.008
              20.0-24.0  0.005
              25.0-29.0  0.008
              30.0-34.0  0.006
              35.0-39.0  0.007
              40.0-44.0  0.006
              45.0-49.0  0.004
              50.0-54.0  0.007
              55.0-59.0  0.007
              60.0-64.0  0.007
              65.0-69.0  0.006
              70.0-74.0  0.006
              75.0-79.0  0.003
              80.0-84.0  0.005
                          -----------------------------------------------------
                          epilepsy prevalence rate
                          -----------------------------------------------------
                    age   rate
                  15-19  0.00

Incidence and prevalence 10 years afterwards should be different

In [4]:
pop.advance(years=duration, nWorkers=nWorkers)

pop.print_outcome_incidence_prevalence(outcomeType=OutcomeType.EPILEPSY)

                          -----------------------------------------------------
                          epilepsy incidence rate (first incidence only)
                          -----------------------------------------------------
                    age   rate
                  15-19  0.002
              15.0-19.0  0.005
              20.0-24.0  0.003
              25.0-29.0  0.005
              30.0-34.0  0.005
              35.0-39.0  0.005
              40.0-44.0  0.005
              45.0-49.0  0.005
              50.0-54.0  0.005
              55.0-59.0  0.005
              60.0-64.0  0.005
              65.0-69.0  0.010
              70.0-74.0  0.010
              75.0-79.0  0.015
              80.0-84.0  0.017
              85.0-89.0  0.026
                          -----------------------------------------------------
                          epilepsy prevalence rate
                          -----------------------------------------------------
                    age   rat

### Epilepsy population characteristics at baseline (using prevalence epilepsy model only)

In [7]:
pf = PersonFilter()
pf.add_filter("person", "epilepsy", lambda x: x.has_outcome(OutcomeType.EPILEPSY))

pop = PopulationFactory.get_nhanes_population(n=popSize, year=year, nhanesWeights=nhanesWeights, personFilters=pf)

pop.print_baseline_summary()

                          Printing a summary of risk factors and default treatments...
                          min      0.25    med     0.75     max    mean     sd
                          -----------------------------------------------------
                    age    18.0    34.0    50.0    63.0    80.0    49.1    17.6
                    hdl    10.0    42.0    51.0    62.0   189.0    53.8    16.1
                    bmi    13.6    24.7    29.3    34.5    86.2    30.4     7.9
                totChol    76.0   159.0   184.0   213.0   446.0   187.8    41.9
                   trig    10.0    63.0    94.0   139.0  2684.0   115.7    94.7
                    a1c     3.8     5.3     5.6     6.1    16.2     5.9     1.2
                    ldl     9.0    87.0   109.0   134.0   357.0   112.4    37.3
                  waist    56.4    88.4   100.1   112.5   178.2   101.2    18.0
             creatinine     0.2     0.7     0.8     1.0    12.7     0.9     0.5
                    sbp    64.7   

### Epilepsy adult lifetime risk

In [15]:
pf = PersonFilter()
pf.add_filter("df", "18yo", lambda x: x[PersonFactory.microsimToNhanes[DynamicRiskFactorsType.AGE.value]]==18)

pop = PopulationFactory.get_nhanes_population(n=popSize, year=year, nhanesWeights=nhanesWeights, personFilters=pf)

pop.print_baseline_summary()

pop.advance(years=durationEndOfTime, nWorkers=nWorkers)

pop.print_outcome_incidence_prevalence(outcomeType=OutcomeType.EPILEPSY)

                          Printing a summary of risk factors and default treatments...
                          min      0.25    med     0.75     max    mean     sd
                          -----------------------------------------------------
                    age    18.0    18.0    18.0    18.0    18.0    18.0     0.0
                    hdl    26.0    39.0    50.0    60.0    91.0    51.4    13.2
                    bmi    15.6    20.5    23.7    27.8    72.6    25.7     7.4
                totChol    92.0   141.0   166.0   182.0   260.0   163.1    27.5
                   trig    26.0    57.0    73.0   123.0   366.0    93.7    52.0
                    a1c     4.5     5.0     5.2     5.4     6.3     5.2     0.3
                    ldl    39.0    81.0    94.0   113.0   174.0    96.0    23.1
                  waist    63.6    74.2    82.0    95.0   178.2    86.7    17.9
             creatinine     0.4     0.7     0.8     0.9     1.2     0.8     0.2
                    sbp    84.0   

### Global burden of disease idiopathic epilepsy new cases

In [22]:
#https://vizhub.healthdata.org/gbd-compare/
#advanced settings, choose epilepsy as the cause ad measure either incidence or prevalence

newCasesGBD = {"15-19": 43.78, #new cases per 100,000
               "20-24": 37.43,
               "25-29": 32.37,
               "30-34": 28.43,
               "35-39": 26.08,
               "40-44": 24.61,
               "45-49": 22.87,
               "50-54": 22.8,
               "55-59": 24.65,
               "60-64": 26.11,
               "65-69": 24.71,
               "70-74": 26.66,
               "75-79": 32.53,
               "80-84": 39.28,  
               "85-89": 46.61,  
               "90-94": 60.85,  
               "95-99": 85.17}

print(" "*19, "age", "  rate")
for group, rate in newCasesGBD.items():
            print(f"{group:>23} {rate/100000:6.5f}")

                    age   rate
                  15-19 0.00044
                  20-24 0.00037
                  25-29 0.00032
                  30-34 0.00028
                  35-39 0.00026
                  40-44 0.00025
                  45-49 0.00023
                  50-54 0.00023
                  55-59 0.00025
                  60-64 0.00026
                  65-69 0.00025
                  70-74 0.00027
                  75-79 0.00033
                  80-84 0.00039
                  85-89 0.00047
                  90-94 0.00061
                  95-99 0.00085


### Global burden of disease idiopathic epilepsy prevalence

In [21]:
prevalenceGBD = {"15-19": 350.08, #prevalence rates per 100,000
                 "20-24": 338.18,
                 "25-29": 314.85,
                 "30-34": 285.74,
                 "35-39": 264.1,
                 "40-44": 254.15,
                 "45-49": 241.94,
                 "50-54": 238.33,
                 "55-59": 263.56,
                 "60-64": 322.5,
                 "65-69": 372.46,
                 "70-74": 418.66,
                 "75-79": 483.75,
                 "80-84": 561.57,  
                 "85-89": 649.94,  
                 "90-94": 767.18,  
                 "95-99": 951.51}

print(" "*19, "age", "  rate")
for group, rate in prevalenceGBD.items():
            print(f"{group:>23} {rate/100000:6.5f}")

                    age   rate
                  15-19 0.00350
                  20-24 0.00338
                  25-29 0.00315
                  30-34 0.00286
                  35-39 0.00264
                  40-44 0.00254
                  45-49 0.00242
                  50-54 0.00238
                  55-59 0.00264
                  60-64 0.00323
                  65-69 0.00372
                  70-74 0.00419
                  75-79 0.00484
                  80-84 0.00562
                  85-89 0.00650
                  90-94 0.00767
                  95-99 0.00952


### Trial with a US sample

In [5]:
%%time

pf = None
td = NhanesTrialDescription(trialType = TrialType.COMPLETELY_RANDOMIZED,
                            blockFactors=list(),
                            sampleSize = popSize, 
                            duration = duration, 
                            treatmentStrategies = "sprint",
                            nWorkers = nWorkers, 
                            personFilters=pf, 
                            year=year, nhanesWeights=nhanesWeights, distributions=False)

tr = Trial(td)

tr.print_covariate_distributions()

tr.run()
toa = TrialOutcomeAssessorFactory.get_trial_outcome_assessor()
tr.analyze(toa)
print(tr)

tr.print_treatment_strategy_variables_information()

                          Printing covariate information for people still alive...
                          self=treated, alive people count=  100000               other=control, alive people count=  100000  
                          self=treated, unique alive people count=  5823          other=control, unique alive people count=  5816    
                          self                                                    other
                          -----------------------------------------------------   -----------------------------------------------------
                          min      0.25    med     0.75     max    mean     sd     min     0.25    med     0.75     max    mean     sd
                          -----------------------------------------------------   -----------------------------------------------------
                    age    18.0    32.0    47.0    61.0    80.0    47.3    17.8    18.0    32.0    47.0    61.0    80.0    47.3    17.9
                    sbp  

### Trial with the US epilepsy sample

In [ ]:
%%time

pf = PersonFilter()
pf.add_filter("person", "epilepsy", lambda x: x.has_outcome(OutcomeType.EPILEPSY))

td = NhanesTrialDescription(trialType = TrialType.COMPLETELY_RANDOMIZED,
                            blockFactors=list(),
                            sampleSize = popSize, 
                            duration = duration, 
                            treatmentStrategies = "sprint",
                            nWorkers = nWorkers, 
                            personFilters=pf, 
                            year=year, nhanesWeights=nhanesWeights, distributions=False)

tr = Trial(td)

tr.print_covariate_distributions()

tr.run()
toa = TrialOutcomeAssessorFactory.get_trial_outcome_assessor()
tr.analyze(toa)
print(tr)

tr.print_treatment_strategy_variables_information()